In [2]:
# Import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# we'll be using transfer learning in pre-trained MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
# initialize the initial learning rate, number of epochs to train for
INIT_LR = 0.0001
EPOCHS = 20
BS = 32 # Batch Size

In [5]:
DIRECTORY = "dataset"
CATEGORIES = ["with_mask", "without_mask"]

In [6]:
data = []
labels = []

In [7]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        # Append the image after preprocessing and resizing
        data.append(image)
        labels.append(category)

C:\Users\KIIT\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [8]:
# perform one-hot encoding on the labels
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [9]:
# Convert them into numpy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [10]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

In [11]:
# Construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [12]:
# load the MobileNetV2 network without FC layer
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\KIIT\anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [13]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [14]:
# Combine base model and head model to get desired model
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and make them un-trainable
for layer in baseModel.layers:
    layer.trainable = False

In [15]:
# Compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
# Train the head of the network
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
95/95 [==============================] - 537s 6s/step - loss: 0.5469 - acc: 0.7241 - val_loss: 0.4917 - val_acc: 0.7601
Epoch 2/20
95/95 [==============================] - 507s 5s/step - loss: 0.3184 - acc: 0.8711 - val_loss: 0.3673 - val_acc: 0.8409
Epoch 3/20
95/95 [==============================] - 644s 7s/step - loss: 0.2349 - acc: 0.9163 - val_loss: 0.4299 - val_acc: 0.8201
Epoch 4/20
95/95 [==============================] - 535s 6s/step - loss: 0.1914 - acc: 0.9235 - val_loss: 0.4576 - val_acc: 0.8318
Epoch 5/20
95/95 [==============================] - 536s 6s/step - loss: 0.1687 - acc: 0.9344 - val_loss: 0.4070 - val_acc: 0.8553
Epoch 6/20
95/95 [==============================] - 539s 6s/step - loss: 0.1706 - acc: 0.9370 - val_loss: 0.2408 - val_acc: 0.8944
Epoch 7/20
95/95 [==============================] - 529s 6s/step - loss: 0.1405 - acc: 0.9483 - val_loss: 0.3605 - val_a

In [ ]:
# Make predictions on the testing set
predIdxs = model.predict(testX, batch_size=BS)

# For each image get the class with highest probability
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
# Show a nicely formatted classification report using sklearn
from sklearn.metrics import classification_report
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

In [ ]:
# Save the model to disk
model.save("mask_detector.model", save_format="h5")

In [ ]:
# Plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")